# Увод и опис проблема

У овој вежби ћемо научити како се већ истрениран модел на неком сету података може адаптирати на други сет података.

**У задацима попунити само део кода означен са 3 тарабе (###).**

функцијa **image_loader** која учитава слику са задате локације **path**, трансформише копију слике из BGR (0-255) формата који користи OpenCV у RGB (0.0 - 1.0),а затим враћа **Tensor** слике.

In [ ]:
import torch
import cv2
import typing

In [ ]:
def image_loader(path: str) -> torch.Tensor:
    image = (cv2.imread(path).astype("float32") / 255.0)[:, :, ::-1].copy()
    return torch.from_numpy(image.transpose(2, 0, 1))

Имплементирати callable класе **RandomGamma**.
<ul>
    <li>RandomGamma - Ствара се са аргументом <em>random_gamma_delta</em>. Када се позове примењује гама филтер са вредношћу која има униформну расподелу у интервалу [1 - <em>random_gamma_delta</em> , 1 + <em>random_gamma_delta</em> ]</li>
</ul>

In [ ]:
import numpy as np
class ClipImage():
    def __call__(self, image):
        return np.clip(image, 0.0, 1.0)

    def __repr__(self):
        return 'ClipImage()'

class RandomGamma():
    def __init__(self, random_gamma_delta):
        self.gamma_range = 1.0 - random_gamma_delta, 1.0 + random_gamma_delta

    def __call__(self, image):
        # vraca sliku podignutu na stepen gama, koji se uniformno sempluje iz intervala self.gamma_range
        return 

    def __repr__(self):
        return 'RandomGamma('+str(self.gamma_range)+')'

In [ ]:
def getRandomGamma(randaom_gamma_delta : float) -> torch.Tensor:
    return RandomGamma(randaom_gamma_delta)

def getClipImage() -> torch.Tensor:
    return ClipImage()

In [ ]:
import torchvision

In [ ]:
def compose_transforms():
    return torchvision.transforms.Compose([
    RandomGamma(0.3),
    ClipImage(),
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])


Даље, учитавамо сет података.

In [ ]:
dataset_path = 'dataset/EuroSAT/2750'
dataset = torchvision.datasets.DatasetFolder(root=dataset_path, loader=image_loader, transform=compose_transforms(), extensions="jpg")

In [ ]:
def split_data(ratios : list, dataset: torch.utils.data.DataLoader) -> list:
    DATASET_SEED = 12345
    torch_generator = torch.Generator().manual_seed(DATASET_SEED)
    dataset_size = len(dataset)
    sizes = [int(len(dataset) * ratio) for ratio in ratios[:-1]]
    sizes.append(len(dataset)-sum(sizes))
    return torch.utils.data.random_split(
        dataset, 
        sizes, 
        generator=torch.Generator().manual_seed(DATASET_SEED))

In [ ]:
train_dataset, val_dataset, test_dataset = split_data([0.7, 0.15, 0.15], dataset)

In [ ]:
def getDataLoader(dataset : torch.utils.data.Dataset , batch_size : int) -> torch.utils.data.DataLoader:
    return torch.utils.data.DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=2, 
        drop_last=True, 
        pin_memory=True)

In [ ]:
BATCH_SIZE = 32

train_loader = getDataLoader(train_dataset, BATCH_SIZE)
val_loader = getDataLoader(val_dataset, BATCH_SIZE)

Допунити следећу ћелију!

In [ ]:
from torch import nn

def get_transfer_learning_model():
    model = torchvision.models.resnet50(pretrained=True)
    num_features = model.fc.in_features
    # implementirati poslednji sloj ResNet50 modela da radi sa nas custom dataset
    model.fc = 
    return model

Даље, пребацујемо мрежу на Графичку картицу због бржег тренирања.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = get_transfer_learning_model()
net.to(device)

Допунити код испод тако да критеријум буде **Cross Entropy Loss**, а да оптимизатор буде **Stohastic Gradient Descent** са **learning rate = 0.001**, a **momentum = 0.9**

In [ ]:
###
criterion = 
optimizer = 

In [ ]:
from sklearn.metrics import accuracy_score

def measure_quality(model : nn.Sequential, loader : torch.utils.data.DataLoader, device : any, max_batches: int=None) -> float:
    model.eval()
    iteration_cnt = 0

    all_preds = list()
    all_labels = list()

    with torch.no_grad():
        for i, data in enumerate(loader):
            if max_batches is not None and iteration_cnt == max_batches:
                break

            inputs, labels = data[0].to(device), data[1].to(device)

            outputs = model(inputs)
            _, pred = torch.max(outputs, 1)

            all_preds += list(pred.data.cpu().numpy())
            all_labels += list(labels.data.cpu().numpy())

            iteration_cnt += 1

    model.train()

    return accuracy_score(all_labels, all_preds)

In [ ]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            val_accuracy = measure_quality(net, val_loader, device)
            print(f"epoch {epoch + 1} iter {i + 1} loss: {running_loss / 100} val accuracy: {val_accuracy}")
            running_loss = 0.0

print('Finished Training')

In [ ]:
torch.save(net.state_dict(), 'results/resnet_50_land_use.pt')